In [0]:
%pip install --upgrade databricks-sdk==0.90.0 osdatahub==1.3.4
%restart_python

In [0]:
%sh apt-get update && apt-get install -y gdal-bin libgdal-dev postgis

In [0]:
import os
import subprocess
import glob

# Local temp directory on the driver
local_dir = "/tmp/vector_demo"
os.makedirs(local_dir, exist_ok=True)

srid = "27700"                 # adjust if using a different GeoTIFF
catalog = "catalog_stuarlqxck_tcgppf"
lakehouse_schema = "raster"
imagery_volume = "imagery"

os.environ["CATALOG"] = catalog
os.environ["SCHEMA"] = lakehouse_schema

lakebase_database = "vector"
lakebase_schema = "public"            # your Lakebase schema
table = "contours"       # table to create

os.environ["LAKEBASE_DATABASE"] = lakebase_database
os.environ["LAKEBASE_SCHEMA"] = lakebase_schema
os.environ["TABLE"] = table

In [0]:
from osdatahub import OpenDataDownload

OpenDataDownload.all_products()

In [0]:
contours = OpenDataDownload("Terrain50")

file_name = "terr50_gpkg_gb.zip"

contours.download(file_name=file_name, output_dir=local_dir)

In [0]:
zip_path = os.path.join(local_dir, file_name)
import zipfile

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(local_dir)

In [0]:
%sh ls /tmp/vector_demo/Data/terr50_gb.gpkg

In [0]:
os.environ["INPUT_PATH"] = os.path.join(local_dir, "Data", "terr50_gb.gpkg")

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.postgres import RequestedClaims, RequestedResource

w = WorkspaceClient()

project_id = "61ad7f1c-95f4-4143-8b66-000942afcb89"
branch_id = "br-odd-silence-e1b8my4j"
endpoint_id = "ep-blue-mouse-e1jj0xrm"
endpoint = f"projects/{project_id}/branches/{branch_id}/endpoints/{endpoint_id}"

In [0]:
os.environ["CURRENT_USER"] = w.current_user.me().emails[0].value

In [0]:
host = w.postgres.get_endpoint(endpoint).status.hosts.host
os.environ["LAKEBASE_HOST"] = host

In [0]:
creds = w.postgres.generate_database_credential(endpoint)
os.environ["PGPASSWORD"] = creds.token

In [0]:
%sh which ogr2ogr

In [0]:
%sh ogr2ogr 

In [0]:
pg_conn_str = f"postgresql://{os.environ['CURRENT_USER']}:{os.environ['PGPASSWORD']}@{os.environ['LAKEBASE_HOST']}:5432/{os.environ['LAKEBASE_DATABASE']}?sslmode=require"

In [0]:
%sh psql "postgresql://$LAKEBASE_HOST:5432/databricks_postgres?sslmode=require" -U $CURRENT_USER -c 'create database vector;'

In [0]:
%sh psql "postgresql://$LAKEBASE_HOST:5432/$LAKEBASE_DATABASE?sslmode=require" -U $CURRENT_USER -c 'CREATE EXTENSION IF NOT EXISTS postgis;'

In [0]:
%sh ogr2ogr -f "PostgreSQL" PG:"host=$LAKEBASE_HOST port=5432 user=$CURRENT_USER dbname=$LAKEBASE_DATABASE sslmode=require" $INPUT_PATH